In [1]:
import cv2
import numpy as np
from scipy.linalg import svd, inv, eig
import math
np.set_printoptions(threshold=np.inf)

In [2]:
FACE_PATH = "att_faces/orl_faces" # can be ambiguous
PERSON_NUM = 40
PERSON_FACE_NUM = 10

In [3]:
# 每个人的第一张图像
raw_img = []
# 数据集
data_set = []
# 数据集标签
data_set_label = []

In [4]:
def read_data():
    for i in range(1, PERSON_NUM + 1):
        person_path = FACE_PATH + '/s' + str(i)
        person = []
        person_label = []
        for j in range(1, PERSON_FACE_NUM + 1):
            img = cv2.imread(person_path + '/' + str(j) + '.pgm')
            if j == 1:
                raw_img.append(img)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            height, width = img_gray.shape
            img_col = img_gray.reshape(height * width)
            person.append(img_col)
            person_label.append(i)
        data_set.append(person)
        data_set_label.append(person_label)
    return height, width

In [5]:
# Import Data
height, width = read_data()
X = np.array(data_set)
Y = np.array(data_set_label)
n_person, n_face, n_feature = X.shape
raw_img = np.hstack(raw_img)

In [6]:
miu = X.mean(axis=1)
print(miu.shape)
miu_all = X.reshape(-1,X.shape[-1]).mean(axis=0)
print(miu_all.shape)

(40, 10304)
(10304,)


In [7]:
temp = X - miu.reshape(miu.shape[0],1,miu.shape[1])

In [8]:
sw = np.zeros((temp.shape[-1],temp.shape[-1]))
for i in range(temp.shape[0]):
    sw += np.matmul(temp[i].transpose(),temp[i])

In [9]:
temp = miu - miu_all
sb = 10*np.matmul(temp.transpose(),temp)

In [10]:
s = np.matmul(inv(sw),sb)

In [11]:
w, v = eig(s)

In [ ]:
v

In [6]:
miu = X.mean(axis=0)
x_minus_miu = X-miu

In [7]:
U,sigma,VT = svd(x_minus_miu)

In [11]:
VT[10000]

array([ 1.50061277e-02,  8.40329563e-03,  6.69866877e-03,  5.51508489e-03,
        6.02741360e-03, -3.75587511e-03, -3.20586783e-03,  3.11395211e-03,
        7.89493358e-04,  1.48589187e-03, -4.94231665e-04,  2.86456690e-03,
        6.26731377e-04,  5.50959655e-03, -2.07870215e-03,  5.03479700e-03,
       -1.13248730e-03,  1.40485850e-03, -1.85211234e-02,  2.05307408e-03,
       -1.97362530e-03,  7.45067453e-03,  1.66988064e-03,  1.68839351e-03,
       -1.17655572e-03, -8.07113846e-04,  4.88649406e-03,  1.07001501e-02,
       -1.05998093e-02, -6.40258528e-03, -7.90501673e-04,  1.12641930e-04,
        9.61848796e-04,  1.81728190e-02, -1.79332580e-04, -4.61102650e-03,
       -1.13586909e-02, -3.54379327e-03,  9.73604871e-03,  1.64264966e-03,
        6.50290234e-03,  1.82160119e-03,  7.83594319e-03, -5.13686340e-04,
        1.32647040e-02,  3.74795329e-03,  2.97283691e-03,  4.72819669e-03,
        3.79953290e-03, -1.83170346e-03, -1.19716936e-02, -4.49433910e-03,
       -7.69976937e-03, -

In [12]:
def show_K_eig_faces(k):
    # 取前k个特征向量
    eig_faces = VT[:k]
    
    # 将特征向量放缩到可imshow格式
    max_min = (eig_faces.max(axis=1)-eig_faces.min(axis=1))[:,np.newaxis]
    eig_faces = (eig_faces-eig_faces.min(axis=1)[:,np.newaxis])*255/max_min
    eig_faces = eig_faces.astype(np.uint8)
    
    # 组合
    eig_faces = np.vsplit(eig_faces,eig_faces.shape[0])
    eig_faces = [x.reshape(112,92) for x in eig_faces]
    
    temp = []
    # 每行显示的图像数量
    if k < 10:
        img_per_row = k
        eig_faces = np.hstack(eig_faces)
    else:
        img_per_row = 10
        for i in range(len(eig_faces)//img_per_row+(len(eig_faces)%img_per_row>0)):
            if i != len(eig_faces)//img_per_row:
                temp.append(np.hstack(eig_faces[i*img_per_row:(i+1)*img_per_row]))
            else:
                t1 = eig_faces[i*img_per_row:]
                t2 = [np.zeros((112,92*(img_per_row-len(eig_faces)%img_per_row)),dtype=np.uint8)]
                t2[0][:] = 255
                temp.append(np.hstack(t1+t2))
        eig_faces = np.vstack(temp)
    
    cv2.imshow("principal faces",eig_faces)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
def show_transformed_faces(k):
    # 降维并逆变换
    principals = VT[:k]
    x_hat = np.matmul(principals, x_minus_miu.transpose())
    x_hat = np.matmul(principals.transpose(),x_hat)
    x_hat = (x_hat + miu[:,np.newaxis]).transpose()
    
    # 放缩，组合
    transformed_faces = [x_hat[i*PERSON_FACE_NUM].reshape(112,92) for i in range(PERSON_NUM)]
    transformed_faces = [((x-x.min())*255/(x.max()-x.min())).astype(np.uint8) for x in transformed_faces]
    transformed_faces = np.hstack(transformed_faces)
    
    cv2.imshow("original faces",raw_img)
    cv2.imshow("PCA transformed faces",transformed_faces)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [15]:
def run(k=100):
    show_K_eig_faces(k)
    show_transformed_faces(k)

In [16]:
run(500)